<a href="https://colab.research.google.com/github/MsSaidat25/Labs-Data-Mining/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

library(dplyr)
library(tidyr)

# --- Input data from PDF ---
df_counts <- tribble(
  ~Department, ~Class,   ~Count,
  "Sales",     "Senior", 30,
  "Sales",     "Junior", 80,
  "Systems",   "Senior", 8,
  "Systems",   "Junior", 23,
  "Marketing", "Senior", 10,
  "Marketing", "Junior", 4,
  "Secretary", "Senior", 4,
  "Secretary", "Junior", 6
)

# --- Helper functions ---
entropy_from_counts <- function(counts_vec) {
  p <- counts_vec / sum(counts_vec)
  p <- p[p > 0]
  -sum(p * log2(p))
}

info_gain <- function(df_counts, target_col, attr_col) {
  overall <- df_counts |>
    group_by(.data[[target_col]]) |>
    summarise(n = sum(Count), .groups = "drop")
  info_D <- entropy_from_counts(overall$n)

  totals <- sum(df_counts$Count)
  cond <- df_counts |>
    group_by(.data[[attr_col]]) |>
    summarise(
      H = entropy_from_counts(Count),
      weight = sum(Count) / totals,
      .groups = "drop"
    )
  info_attr <- sum(cond$H * cond$weight)

  list(
    info_D = info_D,
    info_attr = info_attr,
    gain = info_D - info_attr,
    details = cond
  )
}




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# --- Run analysis ---
res <- info_gain(df_counts, "Class", "Department")

cat(sprintf("Info D = %.4f bits\n", res$info_D))
cat(sprintf("Info Department = %.4f bits\n", res$info_attr))
cat(sprintf("Information Gain Department = %.5f bits\n\n", res$gain))

print(
  res$details |>
    mutate(contribution = H * weight) |>
    select(Department, H, weight, contribution)
)

Info D = 0.8990 bits
Info Department = 0.8504 bits
Information Gain Department = 0.04861 bits

# A tibble: 4 × 4
  Department     H weight contribution
  <chr>      <dbl>  <dbl>        <dbl>
1 Marketing  0.863 0.0848       0.0732
2 Sales      0.845 0.667        0.564 
3 Secretary  0.971 0.0606       0.0588
4 Systems    0.824 0.188        0.155 
